In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.animation import FuncAnimation

from swarm_sim import *

%matplotlib inline 

In [ ]:
PATH = '..\data\Traces_Nanosatellites\\track_'
satellites = {}

for i in range(0,100):
    df = pd.read_csv(PATH+str(i)+'.csv', sep=',', header=0)
    df['coords'] = ['x','y','z']
    satellites[i] = df.set_index('coords', drop=True)
    
DURATION = 2000
NB_NODES = 100

satellites[0].head()

In [ ]:
swarm_data = {}

with tqdm(total = DURATION, desc = 'Converting to Swarm') as pbar:
    for t in range(DURATION):
        swarm_data[t] = Swarm(
            connection_range=30000, 
            nodes=[Node(id, node[str(t)].x, node[str(t)].y, node[str(t)].z) for id,node in satellites.items()]
            )
        pbar.update(1)

print(swarm_data[0])

In [ ]:
def compute_neighbor_matrix(swarm_chunk, connection_range=None):
    neighbor_matrix = {}
    with tqdm(total=len(swarm_chunk.keys()), desc='Computing Neighbor matrix') as pbar:
        for t,swarm in swarm_chunk.items():
            neighbor_matrix[t] = swarm.neighbor_matrix(connection_range)
            pbar.update(1)
    return neighbor_matrix

def compute_swarm_degree(swarm_chunk):
    swarm_degree = {}
    with tqdm(total=len(swarm_chunk.keys()), desc='Computing Swarm degree') as pbar:    
        for t,swarm in swarm_chunk.items():
            swarm_degree[t] = swarm.degree()
            pbar.update(1)
    return swarm_degree

def init_network(swarm_chunk, cr=None):
    if not cr:
        cr = swarm_chunk[0].connection_range
    print('\nConnection range:', cr)
    neighbor_matrix = compute_neighbor_matrix(swarm_chunk, connection_range=cr)
    swarm_degree = compute_swarm_degree(swarm_chunk)
    sum_data = [e for t in swarm_degree.keys() for e in swarm_degree[t]]
    print('Minimum number of neighbors:', np.min(sum_data))
    print('Maximum number of neighbors:', np.max(sum_data))
    print('Average number of neighbors:', np.mean(sum_data))
    node_ict = []
    for i in range(NB_NODES):
        node_data = [swarm_degree[t][i] for t in swarm_degree.keys()]
        node_ict.append(node_data.count(0) / float(len(swarm_chunk.keys()))*100)
    mean_ict = np.mean(node_ict)
    print('Mean inter-contact time:', mean_ict)

In [ ]:
init_network(swarm_data)

## Animate connectivity evolution (edges)

In [ ]:
edges = {}

for t, swarm in swarm_data.items():
    edges[t] = []
    for node in swarm.nodes:
        for n in node.neighbors:
            edge = ([node.x, n.x], [node.y, n.y], [node.z, n.z])
            op = ([n.x, node.x], [n.y, node.y], [n.z, node.z])
            if op not in edges[t]:
                edges[t].append(edge)

In [ ]:
# Static image
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
t = 0

swarm = swarm_data[t]
x_data = [node.x for node in swarm.nodes]
y_data = [node.y for node in swarm.nodes]
z_data = [node.z for node in swarm.nodes]
ax.scatter(x_data, y_data, z_data, c='blue', s=50)

for edge in edges[t]:
    ax.plot(edge[0], edge[1], edge[2], c='red')

In [ ]:
def animate(t):
    swarm = swarm_data[t]
    x_data = [node.x for node in swarm.nodes]
    y_data = [node.y for node in swarm.nodes]
    z_data = [node.z for node in swarm.nodes]
    ax.clear()
    time = t*10
    ax.set_title(f'T = {time} sec')
    ax.scatter(x_data, y_data, z_data, c='blue', s=50)
    for edge in edges[t]:
        ax.plot(edge[0], edge[1], edge[2], c='red')

fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')

x_data = []
y_data = []
z_data = []
    
anim = FuncAnimation(fig, animate, frames=len(swarm_data.keys()), interval=100, repeat=False)
anim.save('../output/animations/swarm_edges_2000.png')